In [33]:
from pathlib import Path
from datetime import date, datetime, timedelta
import csv
import json
import configparser

import requests
import psycopg2
from psycopg2 import sql

import numpy as np
import pandas as pd

In [2]:
DATA_PATH = Path('../data')
CONFIG_PATH = Path('../config')

In [3]:
config = configparser.ConfigParser()
config.read(CONFIG_PATH/'test_sources.ini')

['../config/test_sources.ini']

In [15]:
CSV_URL = config['DEFAULT']['CSV_URL']
JSON_URL = config['DEFAULT']['JSON_URL']
SHARED_DB_CON = config['DEFAULT']['SHARED_DB_CON']
PRIVATE_DB_CON = config['DEFAULT']['PRIVATE_DB_CON']

**csv**

In [16]:
def download_file(url, dir_path):
    file_name = url.split('/')[-1]
    file_path = dir_path/file_name

    with requests.get(url) as r:
        r.raise_for_status()
        
        with open(file_path, 'w') as f:
            f.write(r.content.decode('utf-8'))
            
    return file_path

file_path = download_file(CSV_URL, DATA_PATH)
file_path

PosixPath('../data/orders.csv')

In [17]:
!head -2 {file_path}

id заказа,uuid заказа,название товара,дата заказа,количество,ФИО,email
76921,7d9a28f7-418b-4e8a-9ab3-68bf5deba724,Банк триста плод сынок неудобно поезд неожиданный.,2020-05-15 8:31:05,3,Елизавета Семеновна Юдина,belovaanna@mail.ru


In [22]:
def clean_orders_data(file_path):
    clean_file_path = file_path.parent/f'{file_path.stem}_clean.csv'
    field_names = ('order_id', 'order_uuid', 'good_title', 
                   'date', 'amount', 'name', 'email')
    selected_field_names = ('order_uuid', 'good_title', 'date', 
                            'amount', 'name', 'email')

    with open(file_path) as f:
        next(f)
        reader = csv.DictReader(f, fieldnames=field_names)

        with open(clean_file_path, 'w') as f_clean:
            writer = csv.DictWriter(f_clean, 
                                    fieldnames=selected_field_names, 
                                    extrasaction='ignore')
            writer.writeheader()
            
            for line in reader:
                writer.writerow(line)
                
    return clean_file_path

clean_file_path = clean_orders_data(file_path)
clean_file_path

PosixPath('../data/5ed7391379382f568bd22822_clean.csv')

In [19]:
!head -2 {clean_file_path}

order_uuid,good_title,date,amount,name,email
7d9a28f7-418b-4e8a-9ab3-68bf5deba724,Банк триста плод сынок неудобно поезд неожиданный.,2020-05-15 8:31:05,3,Елизавета Семеновна Юдина,belovaanna@mail.ru


**json**

In [24]:
file_path = download_file(JSON_URL, DATA_PATH)
file_path

PosixPath('../data/5ed7391379382f568bd22822')

In [25]:
def clean_status_data(file_path):
    with open(file_path) as f:
        status_data = json.loads(f.read())

    clean_file_path = file_path.parent/f'{file_path.stem}_clean.csv'
    selected_field_names = ('order_uuid', 'payment_status')

    with open(clean_file_path, 'w') as clean_f:
        writer = csv.DictWriter(clean_f, fieldnames=selected_field_names)

        writer.writeheader() 

        for order_uuid, status in status_data.items():
            payment_status = 'success' if status['success'] is True else 'failure'

            status_row = {
                'order_uuid': order_uuid,
                'payment_status': status['success']
            }

            writer.writerow(status_row)
            
    return clean_file_path
            
clean_file_path = clean_status_data(file_path)
clean_file_path

PosixPath('../data/5ed7391379382f568bd22822_clean.csv')

In [26]:
!head -2 {clean_file_path}

order_uuid,payment_status
7d9a28f7-418b-4e8a-9ab3-68bf5deba724,True


**Shared DB**

In [28]:
dsn = 'dbname=postgres user=shop password=1ec4fae2cb7a90b6b25736d0fa5ff9590e11406 host=109.234.36.184 port=5432'
conn = psycopg2.connect(dsn)
cur = conn.cursor()

table_name = sql.Identifier('customers')
sql_query = sql.SQL('SELECT * FROM {}').format(table_name)
cur.execute(sql_query)
customers = cur.fetchall()

table_name = sql.Identifier('goods')
sql_query = sql.SQL('SELECT * FROM {}').format(table_name)
cur.execute(sql_query)
goods = cur.fetchall()

print(customers[:5])
print(goods[:5])

cur.close()
conn.close()

[(97766, 'Нинель Васильевна Носова ', datetime.date(1975, 7, 19), 'F', 'belovaanna@mail.ru'), (96374, 'Валерия Борисовна Фомина\t', datetime.date(1962, 1, 12), 'F', 'doroninleon@rao.com'), (38920, 'Ладислав Аверьянович Щукин', datetime.date(1933, 5, 16), 'M', 'doroninatatjana@rambler.ru'), (62940, 'Анна Кирилловна Беспалова', datetime.date(2005, 8, 30), 'F', 'komarovepifan@yahoo.com'), (44359, 'Ирина Сергеевна Кулагина', datetime.date(1913, 3, 25), 'F', 'kirill2008@zao.com')]
[(25389, 'Банк триста плод сынок неудобно поезд неожиданный.', Decimal('1835.98')), (54759, 'Пропадать беспомощный равнодушный.', Decimal('375.75')), (56751, 'Что мягкий роса научить необычный домашний командование настать.', Decimal('3354.86')), (78120, 'Товар назначить медицина секунда увеличиваться.', Decimal('5270.2')), (85971, 'Скрытый наступать предоставить невозможно посвятить хозяйка.', Decimal('1765.52'))]


In [34]:
def get_table_data(dsn, table_name, data_path):
    file_path = data_path/f'{table_name}.csv'
    
    conn = psycopg2.connect(dsn)
    cur = conn.cursor()
    
    table_id = sql.Identifier(table_name)
    sql_query = sql.SQL("COPY (SELECT * FROM {}) TO STDOUT WITH CSV HEADER").format(table_id)

    with open(file_path, 'w') as f:
        cur.copy_expert(sql_query, f)
        
    cur.close()
    conn.close()
    
    return file_path

In [35]:
file_path = get_table_data(dsn, 'customers', DATA_PATH)
file_path

PosixPath('../data/customers.csv')

In [36]:
def calculate_age(birth_date, datetime_format):
    DAYS_IN_YEAR = 365.25
    age = (datetime.now() - datetime.strptime(birth_date, datetime_format)) // timedelta(days=DAYS_IN_YEAR)
    return age

datetime_format = '%Y-%m-%d %H:%M:%S'
calculate_age('2020-06-01 3:08:10', datetime_format)

0

In [37]:
def clean_customers_data(file_path):
    clean_file_path = file_path.parent/f'{file_path.stem}_clean.csv'
    field_names = ('id', 'name', 'birth_date', 'gender', 'email')
    selected_field_names = ('email', 'age')
    datetime_format = '%Y-%m-%d'

    with open(file_path) as f:
        next(f)
        reader = csv.DictReader(f, fieldnames=field_names)

        with open(clean_file_path, 'w') as f_clean:
            writer = csv.DictWriter(f_clean, 
                                    fieldnames=selected_field_names, 
                                    extrasaction='ignore')
            writer.writeheader()
            
            for line in reader:
                line['age'] = calculate_age(line['birth_date'], datetime_format)
                writer.writerow(line)
                
    return clean_file_path

clean_customers_data(file_path)

PosixPath('../data/customers_clean.csv')

In [38]:
file_path = get_table_data(dsn, 'goods', DATA_PATH)
file_path

PosixPath('../data/goods.csv')

In [39]:
def clean_goods_data(file_path):
    clean_file_path = file_path.parent/f'{file_path.stem}_clean.csv'
    field_names = ('id', 'good_title', 'price')
    selected_field_names = ('good_title', 'price')
    datetime_format = '%Y-%m-%d'

    with open(file_path) as f:
        next(f)
        reader = csv.DictReader(f, fieldnames=field_names)

        with open(clean_file_path, 'w') as f_clean:
            writer = csv.DictWriter(f_clean, 
                                    fieldnames=selected_field_names, 
                                    extrasaction='ignore')
            writer.writeheader()
            
            for line in reader:
                writer.writerow(line)
                
    return clean_file_path

clean_goods_data(file_path)

PosixPath('../data/goods_clean.csv')

**Shared DB**

In [261]:
shared_con = create_engine(SHARED_DB_CONN_URL)

In [262]:
customers_df = pd.read_sql_table('customers', shared_con)
customers_df['id'] = customers_df['id'].astype('str')
customers_df.head()

,id,name,birth_date,gender,email
0,97766,Нинель Васильевна Носова,1975-07-19,F,belovaanna@mail.ru
1,96374,Валерия Борисовна Фомина\t,1962-01-12,F,doroninleon@rao.com
2,38920,Ладислав Аверьянович Щукин,1933-05-16,M,doroninatatjana@rambler.ru
3,62940,Анна Кирилловна Беспалова,2005-08-30,F,komarovepifan@yahoo.com
4,44359,Ирина Сергеевна Кулагина,1913-03-25,F,kirill2008@zao.com


In [263]:
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 5 columns):
id            889 non-null object
name          889 non-null object
birth_date    889 non-null datetime64[ns]
gender        889 non-null object
email         889 non-null object
dtypes: datetime64[ns](1), object(4)
memory usage: 34.9+ KB


In [264]:
customers_df.describe(include='all')

,id,name,birth_date,gender,email
count,889,889,889,889,889
unique,889,889,879,2,889
top,70246,Милен Богданович Горбунов,1950-03-29 00:00:00,M,mark_53@zao.org
freq,1,1,2,445,1
first,NaN,NaN,1904-06-19 00:00:00,NaN,NaN
last,NaN,NaN,2020-06-03 00:00:00,NaN,NaN


In [265]:
items_df = pd.read_sql_table('goods', shared_con)
items_df.columns = ['id', 'good_title', 'price']
items_df['id'] = items_df['id'].astype('str')
items_df.head()

,id,good_title,price
0,25389,Банк триста плод сынок неудобно поезд неожидан...,1835.98
1,54759,Пропадать беспомощный равнодушный.,375.75
2,56751,Что мягкий роса научить необычный домашний ком...,3354.86
3,78120,Товар назначить медицина секунда увеличиваться.,5270.20
4,85971,Скрытый наступать предоставить невозможно посв...,1765.52


In [266]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895 entries, 0 to 894
Data columns (total 3 columns):
id            895 non-null object
good_title    895 non-null object
price         895 non-null float64
dtypes: float64(1), object(2)
memory usage: 21.1+ KB


In [267]:
items_df.describe(include='all')

,id,good_title,price
count,895,895,895.000000
unique,893,895,NaN
top,37258,Поговорить потрясти мотоцикл.,NaN
freq,2,1,NaN
mean,NaN,NaN,4978.378212
std,NaN,NaN,2891.481750
min,NaN,NaN,17.820000
25%,NaN,NaN,2502.935000
50%,NaN,NaN,4889.170000
75%,NaN,NaN,7475.395000


**Prepare_data**

In [268]:
customers_df['age'] = (pd.datetime.now() - customers_df['birth_date']).div(pd.Timedelta(value=1, unit='Y')).astype('int')
customers_df['age'].describe()

/home/jupyter/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: M and Y units are deprecated and will be removed in a future version.
  """Entry point for launching an IPython kernel.


count    889.000000
mean      56.901012
std       34.184471
min        0.000000
25%       27.000000
50%       58.000000
75%       87.000000
max      115.000000
Name: age, dtype: float64

In [269]:
customers_df['name'] = customers_df['name'].str.strip()
customers_df['email'] = customers_df['email'].str.strip()

In [270]:
orders_df['name'] = orders_df['name'].str.strip()
orders_df['email'] = orders_df['email'].str.strip()

In [271]:
status_df['payment_status'] = np.where(status_df['success'], 'success', 'fail')

**Merge data**

In [276]:
orders_cols = ['order_uuid', 'good_title', 'date', 'amount', 'name', 'email']
status_cols = ['order_uuid', 'payment_status']
customers_cols = ['email', 'age']
items_cols = ['good_title', 'price']

In [277]:
full_df = orders_df[orders_cols].merge(status_df[status_cols], on='order_uuid', right_index=True)
full_df = full_df.merge(customers_df[customers_cols], how='left', on='email')
full_df = full_df.merge(items_df[items_cols], how='left', on='good_title')

full_df['total_price'] = full_df.eval('amount * price')

full_df

,order_uuid,good_title,date,amount,name,email,payment_status,age,price,total_price
0,7d9a28f7-418b-4e8a-9ab3-68bf5deba724,Банк триста плод сынок неудобно поезд неожидан...,2020-05-15 8:31:05,3,Елизавета Семеновна Юдина,belovaanna@mail.ru,success,44.0,1835.98,5507.94
1,f70a0b8c-f6cd-4876-b21b-33ee76d2d993,Пропадать беспомощный равнодушный.,2020-06-01 3:08:10,5,Ершов Милий Григорьевич,doroninleon@rao.com,fail,58.0,375.75,1878.75
2,5507582b-992f-4eb0-97e6-6d4b5d66b06c,Что мягкий роса научить необычный домашний ком...,2020-05-12 21:50:55,4,Маслов Твердислав Фадеевич,doroninatatjana@rambler.ru,success,87.0,3354.86,13419.44
3,c48a1e2e-06cf-4cb5-b697-0c695227b450,Товар назначить медицина секунда увеличиваться.,2020-05-29 7:17:52,3,Анисим Харитонович Лазарев,komarovepifan@yahoo.com,success,14.0,5270.20,15810.60
4,1d83bf09-4c3a-458f-91c3-bb27c475ede8,Скрытый наступать предоставить невозможно посв...,2020-05-09 17:07:10,3,Силина Элеонора Филипповна,kirill2008@zao.com,fail,107.0,1765.52,5296.56
...,...,...,...,...,...,...,...,...,...,...
890,e1225cc6-5f1e-4754-8c6a-d5c40769da1b,Развернуться поймать ручей палка.,2020-05-23 15:18:44,1,Уваров Валерьян Арсеньевич,ratiborkudrjashov@yandex.ru,fail,26.0,9757.25,9757.25
891,746cea7f-0b39-4dd8-8c80-182cc98035d3,Кузнец спасть желание избегать сопровождаться ...,2020-06-01 3:21:56,2,Марфа Геннадиевна Стрелкова,izjaslavkuznetsov@rao.ru,success,43.0,2635.40,5270.80
892,1f3b853d-7ab1-4087-b1a1-757a016eeb05,Белье плод нож карандаш металл правильный поси...,2020-05-19 20:21:47,5,г-н Гаврилов Ладимир Дорофеевич,semenovaevdokija@vishnjakov.biz,fail,12.0,6956.57,34782.85
893,789717a6-5e5b-4434-b724-1f974a6e910a,Единый человечек о лететь эффект смелый смерте...,2020-05-03 18:00:49,5,Евсеев Юлий Владиславович,trifonstrelkov@gmail.com,success,104.0,5308.94,26544.70


In [278]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 895 entries, 0 to 894
Data columns (total 10 columns):
order_uuid        895 non-null object
good_title        895 non-null object
date              895 non-null object
amount            895 non-null int64
name              895 non-null object
email             895 non-null object
payment_status    895 non-null object
age               889 non-null float64
price             895 non-null float64
total_price       895 non-null float64
dtypes: float64(3), int64(1), object(6)
memory usage: 76.9+ KB


In [279]:
cols = ['name', 'age', 'good_title', 'date', 'payment_status', 'total_price', 'amount']

full_df[cols].head()

,name,age,good_title,date,payment_status,total_price,amount
0,Елизавета Семеновна Юдина,44.0,Банк триста плод сынок неудобно поезд неожидан...,2020-05-15 8:31:05,success,5507.94,3
1,Ершов Милий Григорьевич,58.0,Пропадать беспомощный равнодушный.,2020-06-01 3:08:10,fail,1878.75,5
2,Маслов Твердислав Фадеевич,87.0,Что мягкий роса научить необычный домашний ком...,2020-05-12 21:50:55,success,13419.44,4
3,Анисим Харитонович Лазарев,14.0,Товар назначить медицина секунда увеличиваться.,2020-05-29 7:17:52,success,15810.60,3
4,Силина Элеонора Филипповна,107.0,Скрытый наступать предоставить невозможно посв...,2020-05-09 17:07:10,fail,5296.56,3


**Private DB**

In [40]:
private_con = create_engine(PRIVATE_DB_CONN_URL)